In [1]:
import datetime
import numpy as np
import pyproj
import geopyspark as gps

from pyspark import SparkContext
from shapely.geometry import box, Point

In [2]:
conf = gps.geopyspark_conf(master="local[*]", appName="myapp")
pysc = SparkContext(conf=conf)

In [3]:
raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="file:///home/hadoop/notebooks/cropped.tif")

In [4]:
tiled_raster_layer = raster_layer.tile_to_layout(gps.GlobalLayout(), target_crs=3857)
tiled_raster_layer

TiledRasterLayer(layer_type=LayerType.SPATIAL, zoom_level=11, is_floating_point_layer=False)

In [5]:
pyramided_layer = tiled_raster_layer.pyramid()
pyramided_layer.max_zoom

11

In [7]:
for tiled_layer in pyramided_layer.levels.values():
    gps.write(uri="file:///home/hadoop/notebooks/p1", layer_name="cropped", tiled_raster_layer=tiled_layer)

In [18]:
t1 = pyramided_layer.levels.get(11)

In [19]:
t1.save_stitched(path='/home/hadoop/notebooks/l_11.tif')

In [113]:
extent = tiled_raster_layer.layer_metadata.extent
print(extent)

Extent(xmin=8905559.263461886, ymin=557272.7666948338, xmax=9016908.322868723, ymax=781182.2141882492)


In [68]:
print(extent.xmin + (extent.xmax-extent.xmin)/2)
print(extent.ymin + (extent.ymax-extent.ymin)/2)


8961233.793165304
669227.4904415414


In [114]:
tiled_raster_layer.layer_type

<LayerType.SPATIAL: 'spatial'>

In [110]:
max_val = 0.0;
max_x = extent.xmin;
max_y = extent.ymin;
for x in range(int(extent.xmin), int(extent.xmax), 10000):
    for y in range(int(extent.ymin), int(extent.ymax), 10000):
        p1 = Point(x, y)
        v = tiled_raster_layer.get_point_values(points=[p1])
        if (v and v[0][1] and v[0][1][0] > max_val):
            max_x=x
            max_y=y
            max_val=v[0][1][0]

In [111]:
print(max_val)
print(max_x)
print(max_y)

2013.0
8995559
757272


In [116]:
tiled_raster_layer_new = (tiled_raster_layer / 2013)

In [117]:
tiled_raster_layer_new.save_stitched(path='/home/hadoop/notebooks/l2.tif')

In [118]:
p1 = Point(max_x, max_y)
tiled_raster_layer_new.get_point_values(points=[p1])

[(<shapely.geometry.point.Point at 0x7f8595565550>, [1.0])]

In [119]:
'''
A square neighborhood with an extent of 1.
o = source cell
x = cells that fall within the neighbhorhood

x x x
x o x
x x x
'''

square = gps.Square(extent=1)

In [120]:
tiled_raster_layer_new = tiled_raster_layer.focal(operation=gps.Operation.MEAN, neighborhood=square)


In [121]:
p1 = Point(max_x, max_y)
tiled_raster_layer_new.get_point_values(points=[p1])

[(<shapely.geometry.point.Point at 0x7f85954de4e0>, [2028.5555555555557])]